# Demonstrating the effect of AT bias on our BFA data

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as pl
import seaborn as sns
import math

def reverse_complement(seq):
    """reverse complements a dna sequence (does not convert any non-atcg/ATCG characters)"""
    watson_crick = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'a': 't', 't': 'a', 'g': 'c', 'c': 'g'}
    return ''.join([watson_crick.setdefault(c, c) for c in seq[::-1]])

# including the sequence between the barcodes to get the max AT run of the whole region (which is flanked by CG seqs)
middle_seq = 'ATAACTTCGTATAATGTATGCTATACGAAGTTAT'

def gc(s):
    return len([i for i in s if i in ['G', 'C']])
    
def sliding_window_min(row, win_size):
    s = row['Diverse_BC'] + middle_seq + reverse_complement(row['Environment_BC'])
    return min([gc(s[i:i+win_size]) for i in range(len(s)-win_size+1)])

def plot_all_bias_terms(tds, outpng):
    f, subps = pl.subplots(8, len(all_envs), figsize=(7.5,10), sharex=True, sharey=True, dpi=200)
    rc = 0
    b = 0
    for td in tds:
        rep_dict = {e: sorted(set([i.split('-')[2] for i in td.keys() if e in i and 'Time' in i])) for e in all_envs}
        for rep in ['R1', 'R2', 'R3']:
            if len([i for i in all_envs if rep in rep_dict[i]]) > 0:
                for e in range(len(all_envs)):
                    env = all_envs[e]
                    sub = subps[rc][e]
                    tps = sorted([i for i in td.keys() if 'Time' in i and env + '-' + rep in i],
                                         key=lambda x: int(x[x.index('Time')+4:]))
                    times = [int(x[x.index('Time')+4:]) for x in tps]
                    gc_bias_effects = [[] for i in range(1, 7)]
                    for tp in tps:
                        for gc in range(6):
                            gc_bias_effects[gc].append(np.log10(np.mean(td.loc[td['min.lox.GC.w26']==gc+1][tp])/sum(td[tp])))
                    for gc in range(6):
                        sub.plot(times, gc_bias_effects[gc], label=str(gc+1), linewidth=1)
                    if rc == 0:
                        sub.set_title(env, y=1.1, fontsize=6.5)
                    if e == 9:
                        sub.set_ylabel(bfas[b] + ' ' + rep, fontsize=6.5, rotation='horizontal', ha='left', labelpad=20)
                        sub.yaxis.set_label_position("right")
                    if e == 0 and rc == 4:
                        sub.set_ylabel('Log10(freq)', fontsize=9, rotation='horizontal', labelpad=35)
                    if e == 4 and rc == 7:
                        sub.set_xlabel('Generation', fontsize=9, x=1)
                        
                    sub.set_xlim([8,40])
                    sub.tick_params(axis='both', which='major', labelsize=5.5)
                rc += 1
        b += 1
        
    sns.despine()
    leg = subps[0][6].legend(fontsize=5.5, ncol=6, bbox_to_anchor=(1,1.8), title='Min GC bp in 26 bp sliding window in barcode region')
    leg.get_title().set_fontsize('5.5')
    f.savefig(outpng)
    

bfas = ['hBFA1', 'hBFA2', 'dBFA2']
dats = dict()
putative_neuts = dict()
for b in bfas:
    dats[b] = pd.read_csv('../../output/' + b + '/' + b + '_bc_counts_final_w_envs.csv')
putative_neuts['hBFA1'] = list(dats['hBFA1'].loc[dats['hBFA1']['Subpool_environment'] == 'YPD_alpha']['Full_BC'])
putative_neuts['hBFA2'] = list(dats['hBFA2'].loc[dats['hBFA2']['Subpool_environment'] == 'CLM_2N']['Full_BC'])
putative_neuts['dBFA2'] = list(dats['dBFA2'].loc[dats['dBFA2']['Subpool_environment'] == 'YPDAnc_2N'].loc[dats['dBFA2']['Subpool_replicate'] == '-R1_1']['Full_BC'])
plot_rows = {'hBFA1': 2, 'hBFA2': 3, 'dBFA2': 3}
all_envs = ['YPD', 'SC', '37C', 'pH3_8', 'pH7_3', 'FLC4', 'GlyEtOH', 'CLM', '21C', '02M_NaCl']
for b in bfas:
    td = dats[b]
    # Getting the minimum GC content in a 26-bp sliding window
    td['min.lox.GC.w26'] = td.apply(lambda row: sliding_window_min(row, 26), axis=1)

plot_all_bias_terms([dats[b].loc[dats[b]['Full_BC'].isin(putative_neuts[b])] for b in bfas], '../../Figures/supp_figs/all_at_bias.png')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in log10


In [5]:
for b in dats:
    print(b)
    td = dats[b]
    print('Percentage excluded for 4 or less on GC measurement:', len(td[td['min.lox.GC.w26']<=3])/len(td))
    print('Percentage in reads:', np.sum(td[td['min.lox.GC.w26']<=3]['Total_Counts'])/np.sum(td['Total_Counts']))
    

hBFA1
Percentage excluded for 4 or less on GC measurement: 0.0801626488527447
Percentage in reads: 0.029492666643014916
hBFA2
Percentage excluded for 4 or less on GC measurement: 0.018599978937306083
Percentage in reads: 0.06841643617169327
dBFA2
Percentage excluded for 4 or less on GC measurement: 0.19314408141403322
Percentage in reads: 0.06569986204607878


In [10]:
[i for i in pd.read_csv('../../output/' + b + '/' + b + '_bc_counts_final_w_envs.csv') if 'nv' in i]

['Environment_BC', 'Subpool_environment']

In [15]:
for b in dats:
    print(b)
    td = dats[b]
    print(td[td['min.lox.GC.w26']<=3]['Subpool_environment'].value_counts())
    

hBFA1
pH3_8_alpha      131
21C_alpha         26
pH7_3_alpha       25
GlyEtOH_alpha     23
YPD_alpha         16
SC_alpha          16
FLC4_alpha        14
37C_alpha         13
CLM_alpha          8
none               4
Name: Subpool_environment, dtype: int64
hBFA2
none              1601
08M_NaCl_alpha     379
pH3_8_alpha         69
FLC4_alpha          57
02M_NaCl_alpha      37
21C_alpha           35
GlyEtOH_2N          29
CLM_2N              22
FLC4_2N             17
GlyEtOH_alpha       15
pH7_3_alpha         10
37C_alpha            8
SC_alpha             7
CLM_alpha            6
YPD_alpha            4
Name: Subpool_environment, dtype: int64
dBFA2
none           1467
YPD_2N           52
SC_2N            50
02M_NaCl_2N      35
FLC4_2N          35
pH3_8_2N         35
GlyEtOH_2N       22
37C_2N           21
48Hr_2N          20
pH7_3_2N         20
21C_2N           20
YPDAnc_2N        19
CLM_2N            7
Name: Subpool_environment, dtype: int64
